In [11]:
import cv2
import mediapipe as mp

import win32api, win32con
import time
import threading

In [12]:
def getLeftHandLM(result):
    for id, classification in enumerate(result.multi_handedness):
        if classification.classification[0].label == "Left":
            return result.multi_hand_landmarks[id].landmark
    return None

def getRightHandLM(result):
    for id, classification in enumerate(result.multi_handedness):
        print(classification.classification)
        if classification.classification[0].label == "Right":
            return result.multi_hand_landmarks[id].landmark
    return None


In [13]:
def lerp_cursor(x1, y1, x2, y2, t1):
    t0 = time.time()

    def timelerp(a, b):
        if t1 - t0 == 0:
            return (a, b)
        return (a + (time.time() - t0) / (t1 - t0) * (b - a))

    while time.time() < t1:
        pos = (int(timelerp(x1, x2)), int(timelerp(y1, y2)))
        win32api.SetCursorPos(pos)

In [30]:
cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    min_detection_confidence=0.3,
    max_num_hands = 1
)
mpDraw = mp.solutions.drawing_utils

INVERT_X = True
INVERT_Y = False

t0 = time.time()
while True:
    dt = time.time() - t0
    t0 = time.time()
    ret, img = cap.read()
    if ret:
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(imgRGB)
        
        resultMLM = result.multi_hand_landmarks
        if resultMLM:
            for handLms in resultMLM:
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
        
            # Check for right hand palm location
            rhLM = result.multi_hand_landmarks[0].landmark
            if rhLM:
                def scaleMousePosition(x, invert, scale=1.5):
                    if invert:
                        return (1 - x) * scale + (1 - scale) / 2
                    return x * scale + (1 - scale) / 2

                x = int(scaleMousePosition(rhLM[mpHands.HandLandmark.WRIST].x, INVERT_X) * win32api.GetSystemMetrics(0))
                y = int(scaleMousePosition(rhLM[mpHands.HandLandmark.WRIST].y, INVERT_Y) * win32api.GetSystemMetrics(1))

                if not (win32api.GetCursorPos()[0] - x < 20 and win32api.GetCursorPos()[1] - y < 20):
                    # Lerping if far away
                    lerp_thread = threading.Thread(target=lerp_cursor, args=(win32api.GetCursorPos()[0], win32api.GetCursorPos()[1], x, y, t0 + dt))
                    lerp_thread.start()
                # elif win32api.GetCursorPos()[0] - x + win32api.GetCursorPos()[1] - y > 1:
                else:
                    win32api.SetCursorPos((x, y))




            

        cv2.imshow('img', img)

    if cv2.waitKey(1) == ord('i'):
        INVERT_Y = not INVERT_Y
    if cv2.waitKey(1) == ord('o'):
        INVERT_X = not INVERT_X
    if cv2.waitKey(1) == ord('q'):
        break


In [ ]:
print(result.multi_handedness)

[classification {
  index: 0
  score: 0.9928168058395386
  label: "Left"
}
]
